In [156]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [607]:
"""
PREP DATA
"""

from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
from model_training import make_predictions

# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()
# make_predictions()

import yahoo_utils
# yahoo_utils.refresh_players()
# yahoo_utils.refresh_player_games()
# yahoo_utils.refresh_teams()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.login.yahoo.com:443
DEBUG:urllib3.connectionpool:https://api.login.yahoo.com:443 "POST /oauth2/get_token HTTP/11" 200 1155
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 353
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/league/453.l.15482/teams?format=json HTTP/11" 200 2594
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 351
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/team/453.l.15482.t.1;out=metadata,stats,standings,roster,draftresults,matchups?format=json HTTP/11" 200 20976
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format

In [734]:
from model_training import load_predictions
import lineup_utils
import yahoo_utils

        
class TeamAnalytics():
    
    def __init__(self, team_key, mode='season', date=None, reinit_lineup_preds=True):
        self.mode = mode
        
        self.date = date if date is not None else pd.Timestamp.now(tz='America/Los_Angeles').date()
            
        self.matchup = yahoo_utils.get_matchup(team_key=team_key, date=date)            
        self.players = yahoo_utils.load_players(team_key=team_key, matchup=self.matchup)
        self.preds, self.player_stds = load_predictions()
        self.player_games = yahoo_utils.load_player_games(mode=self.mode, date=date, matchup=self.matchup)

        
        # could be populated instead of hardcoded
        self.cats = ['g','a','sog','fow','hit','block','pim','plusmin','ppp', 'ga','win','so','save']
        

        self.selected_team = []        
        self.n_games_lineup = 0
        self.own_totals = pd.Series({c:0 for c in self.cats})
        
        self.rest_games = None
        self.lineup_preds = None
        self.rel_lineup_preds = None
        
        self.make_full_selections()
        if reinit_lineup_preds:
            self.get_lineup_preds(only_rostered=False)

        self.queue = {}
    
    def get_lineup_preds(self, only_rostered=True):        
        stats_available = [p for p in self.players.index.tolist() if p in self.player_games.index]
        stats_available = [p for p in stats_available if p in self.preds.index]        
        if only_rostered:
            stats_available = [p for p in stats_available if p in self.players[self.players.is_rostered].index.tolist()]
        
        player_games = self.player_games.loc[stats_available].reset_index()
            
        self.rest_games = lineup_utils.get_rest_of_period_games(self.date, player_games, self.selected_team, self.players, self.preds)
        stats_available = [p for p in stats_available if p in self.rest_games.index]
        self.lineup_preds = self.preds.loc[stats_available, self.cats].apply(lambda x: x * self.rest_games[stats_available])
        self.lineup_preds['ga'] = self.preds.loc[stats_available, 'ga']
        self.lineup_preds['games_left'] = self.rest_games.loc[stats_available]
        self.lineup_preds = self.lineup_preds.join(self.players[['available']], how='left')
        
        if self.mode == 'week':
            comparison = [p for p in self.players.loc[self.players.opp_lineup].index if p in self.lineup_preds.index]
        else:
            comparison = [p for p in self.players.loc[self.players.is_rostered].index if p in self.lineup_preds.index]
      

        # standard deviation that takes into account individual players' uncertainty
        std_factor =  ( (self.preds.loc[comparison, self.cats].var()) + (self.player_stds / (17**0.5))**2 ) ** 0.5
        lineup_len_divisors = (self.lineup_preds.notna() + self.preds.loc[self.selected_team, self.cats].count()).apply(lambda x: x * (self.n_games_lineup + self.rest_games.mean()) / (len(self.selected_team)+1))

        self.rel_lineup_preds = ((self.lineup_preds.fillna(0) + self.own_totals)/lineup_len_divisors - self.preds.loc[comparison, self.cats].mean()) / std_factor
        self.lineup_preds['rank'] = self.rel_lineup_preds.sum(axis=1)
        self.lineup_preds['games_left'] = self.rest_games.loc[stats_available]
        
        return self
    
    
    def select_player(self, selected_player, only_rostered=False):
        
        self.n_games_lineup += self.rest_games.loc[selected_player]
        self.selected_team.append(selected_player)
        self.own_totals += self.lineup_preds.loc[selected_player, self.cats].astype(float).fillna(0)
        self.own_totals['ga'] = self.preds.loc[self.selected_team, 'ga'].mean()
        
        self.get_lineup_preds(only_rostered=only_rostered)
        return self
    
    def unselect_player(self, selected_player, only_rostered=False):
        self.selected_team.remove(selected_player)
        self.get_lineup_preds(only_rostered=only_rostered)
        
        self.own_totals -= self.lineup_preds.loc[selected_player, self.cats].astype(float).fillna(0)
        self.own_totals['ga'] = self.preds.loc[self.selected_team, 'ga'].mean()
        
        self.n_games_lineup -= self.rest_games.loc[selected_player]
        return self
    
    def render(self):
        return self.players.join(self.lineup_preds, how='inner').sort_values('rank')
    
    def make_full_selections(self):
        team_max_length = 17
        self.select_none()
        self.get_lineup_preds(only_rostered=True)
        ir = self.players.selected_position.fillna('').str.contains('IR') == True
        available = self.players.loc[(self.players.current_lineup)&(~ir)].index.tolist()
        while len(self.selected_team) < team_max_length and len(available) > 0:
            selection = self.lineup_preds.loc[available, 'rank'].idxmax()
            self.select_player(selection, True)
            available = [p for p in available if p not in self.selected_team]
            print(f'Selected {self.players.loc[selection]['name']}')
        
    def select_none(self):
        self.selected_team = []
        self.n_games_lineup = 0
        self.own_totals = pd.Series({c:0 for c in self.cats})
        self.get_lineup_preds()
            
    def add_to_queue(self, player, included):
        self.queue[player] = included

    def execute_queue(self):
        for plr in self.queue:
            include = self.queue(pop)
            if include and plr not in self.selected_team:
                self.select_player(plr, only_rostered=False)
            elif not include and plr in self.selected_team:
                self.unselect_player(plr, only_rostered=False)

def refresh_opp_stats(data):
    selected_teams = []
    team_keys = [p for p in full_data.players.team_key.dropna().unique() if p != team_key]
    for tk in team_keys:
        team = TeamAnalytics(tk, mode=data.mode, date=data.date, reinit_lineup_preds=False)
        stats = team.own_totals.copy()
        stats['n_games'] = data.n_games_lineup
        stats['team'] = tk
        selected_teams.append(stats)
    df = pd.DataFrame(selected_teams)
    df.to_csv('data/opp_stats.csv', index=False)
    
def load_opp_stats():
    return pd.read_csv('data/opp_stats.csv')



In [735]:
team_key = '453.l.15482.t.3'
full_data = TeamAnalytics(team_key, mode='week', date=pd.Timestamp.now().date())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.login.yahoo.com:443
DEBUG:urllib3.connectionpool:https://api.login.yahoo.com:443 "POST /oauth2/get_token HTTP/11" 200 1155
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 354
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/team/453.l.15482.t.3;out=metadata,stats,standings,roster,draftresults,matchups?format=json HTTP/11" 200 21665
Selected Mikko Rantanen
Selected Tage Thompson
Selected Matt Boldy
Selected Steven Stamkos
Selected Sam Bennett
Selected Mark Scheifele
Selected Frank Vatrano
Selected Brandon Montour
Selected Zach Werenski
Selected Thomas Chabot
Selected Shayne Gostisbehere
Selected Nikita Zadorov
Selected John Tavares
Selected Cam Talbot
Selected Sam Montembeault
Selected Charlie Lindgren
Selecte

In [ ]:
refresh_opp_stats(full_data)
stats = load_opp_stats()
stats

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 353
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/team/453.l.15482.t.6;out=metadata,stats,standings,roster,draftresults,matchups?format=json HTTP/11" 200 21589
Selected Vincent Trocheck
Selected Joey Daccord
Selected Kevin Lankinen
Selected Sebastian Aho
Selected Jeremy Swayman
Selected Artemi Panarin
Selected Quinn Hughes
Selected Tom Wilson
Selected Victor Hedman
Selected Auston Matthews
Selected Alex Tuch
Selected Miro Heiskanen
Selected Jason Robertson
Selected Travis Sanheim
Selected Brock Boeser
Selected Joonas Korpisalo
Selected Sebastian Aho
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/